In [ ]:
"""
This is a template for the Physics-Informed Convolution-AutoEncoder (PICAE)
estimator. The algorithm enforces conservaiton of mass; built with PyTorch.

Note: example data is randomly sampled from a normal distribution, 
      hence the results won't be valid.
      
-pikarpov      
"""

%matplotlib inline
import os
import sys

sys.path.append("../../")

from sapsan.lib.backends import MLflowBackend, FakeBackend
from sapsan.lib.data import HDF5Dataset, EquidistantSampling, flatten
from sapsan.lib.estimator.picae.picae_estimator import PICAE, PICAEConfig
from sapsan.lib import Train, Evaluate

In [ ]:
# --- Experiment tracking backend ---
# MLflow - the server will be launched automatically
# in case it won't, type in cmd: mlflow ui --port=5000
# to use it, set mlflow = True

mlflow = False
experiment_name  = "PICAE experiment"

if mlflow: 
    tracking_backend = MLflowBackend(experiment_name, host="localhost", port=5000)
else: tracking_backend = FakeBackend()

In [ ]:
# --- Data setup ---
# In the intereset of loading and training multiple timesteps
# one can specify which checkpoints to use and where
# they appear in the path via syntax: {checkpoint:format}
# 
# Next, you need to specify which features to load; let's assume 
#         path = "{feature}.h5"
# 
#  1. If in different files, then specify features directly;
#     The default HDF5 label will be the last label in the file
#     Ex: features = ['velocity', 'denisty', 'pressure']
#  2. If in the same, then duplicate the name in features
#     and specify which labels to pull
#     Ex: features = ["data", "data", "data"]
#         feature_labels = ['velocity', 'density', 'pressure']

path = "data/picae/t{checkpoint:1.0f}/picae_random_{feature}.h5"

features = ['u']
target   = ['u']

#Dimensionality of your data in format [D,H,W]
INPUT_SIZE = [32,32,32]

#Reduce dimensionality to the following in format [D,H,W]
SAMPLE_TO = [16,16,16]

#Sampler to use for reduction
sampler = EquidistantSampling(SAMPLE_TO)

In [ ]:
#Load the data
data_loader = HDF5Dataset(path        = path,
                          features    = features,
                          target      = target,
                          checkpoints = [0],
                          input_size  = INPUT_SIZE,
                          sampler     = sampler)

x, y    = data_loader.load_numpy()
loaders = data_loader.convert_to_torch([x,y])

In [ ]:
estimator = PICAE(config  = PICAEConfig(nfilters    = 6, 
                                        kernel_size = (3,3,3), 
                                        enc_nlayers = 3,                        
                                        dec_nlayers = 3, 
                                        n_epochs    = 5,
                                        patience    = 10, 
                                        min_delta   = 1e-5),
                  loaders = loaders)

In [ ]:
# Set the experiment
training_experiment = Train(model           = estimator,
                            backend         = tracking_backend,                 
                            data_parameters = data_loader)

# Train the model
estimator = training_experiment.run()

In [ ]:
# --- Test the model ---
# Load the data
data_loader = HDF5Dataset(path        = path,
                          features    = features,
                          target      = target,
                          checkpoints = [0],
                          input_size  = INPUT_SIZE,
                          sampler     = sampler)

x, y    = data_loader.load_numpy()
loaders = data_loader.convert_to_torch([x,y])

# Set the test experiment
# loaders need to take in the same size data as the individual training batch
estimator.loaders = loaders
evaluation_experiment = Evaluate(model           = estimator,
                                 backend         = tracking_backend,
                                 data_parameters = data_loader)


# Test the model
# Evaluation returns a dict, results = {'predict':np.ndarray, 'target':np.ndarray}
results = evaluation_experiment.run()